In [ ]:
import os, shutil, yaml, subprocess

# Labeled Custom Data

## 파일 정리하기

### 파일 리스트 만들기

In [ ]:
png_img_base_path = "/home/erbl/data/MedicalAids/Images_RGB"
train_label_base_path = "/home/erbl/data/MedicalAids/Annotations_RGB"
test_label_base_path = "/home/erbl/data/MedicalAids/Annotations_RGB_TestSet2"

img_list = list(map(lambda x: os.path.splitext(x)[0], os.listdir(png_img_base_path)))
train_label_list = list(map(lambda x: os.path.splitext(x)[0], os.listdir(train_label_base_path)))
test_label_list = list(map(lambda x: os.path.splitext(x)[0], os.listdir(test_label_base_path)))

print(img_list[:10])

In [ ]:
# train, test 비율
total, train, test = len(img_list), len(train_label_list), len(test_label_list)

print("train: ", f"{train/total*100: .2f}", "%")
print("test: ", f"{test/total*100: .2f}", "%")

### 파일 변환 (png to jpg) 및 이동

In [ ]:
yolov4_base_path = "/home/erbl/Git/RealTime-Object-Detection/yolov4-train"
darknet_path = f"{yolov4_base_path}/darknet"
obj_base_path = jpg_img_base_path = f"{darknet_path}/data/obj"

In [ ]:

os.makedirs(yolov4_base_path, exist_ok=True)
os.makedirs(jpg_img_base_path, exist_ok=True)

In [ ]:
# PNG TO JPG
os.makedirs(jpg_img_base_path, exist_ok=True)
for img in img_list:
    subprocess.call(["convert", f"{png_img_base_path}/{img}.png", f"{jpg_img_base_path}/{img}.jpg"])

## train.txt & test.txt 만들기
고민되는 점: 여기서 suffle 한 후 train test split??

In [ ]:
print(yolov4_base_path)
print(jpg_img_base_path)
print(obj_base_path)
print(darknet_path)

In [ ]:
with open(f"{darknet_path}/data/train.txt", "w") as f:
    lines = ""
    for train_filename in train_label_list:
        lines += f"{jpg_img_base_path}/{train_filename}.jpg\n"
    f.write(lines)

In [ ]:
with open(f"{darknet_path}/data/test.txt", "w") as f:
    lines = ""
    for test_filename in test_label_list:
        lines += f"{jpg_img_base_path}/{test_filename}.jpg\n"
    f.write(lines)

## annotation 파일 형식 변환 (yml to txt)

### 새로운 형식의 파일 만들기

In [ ]:
# 클래스 인데싱 딕셔너리
class_dir = {'person': 0, 'wheelchair': 1, 'push_wheelchair': 2, 'crutches': 3, 'walking_frame': 4} # 수정, 추가 필요

In [ ]:
# 한 이미지의 annotation 정보들 바꾸어 리스트로 반환
def transform_annotation(annotations):

		results = []
		if 'object' in annotations['annotation']:

			objs = annotations['annotation']['object']
			height = int(annotations['annotation']['size']['height'])
			width = int(annotations['annotation']['size']['width'])

			for i in range(len(objs)): # i: bndbox 물체들의 인덱스
				xmax = int(objs[i]['bndbox']['xmax'])
				xmin = int(objs[i]['bndbox']['xmin'])
				ymax = int(objs[i]['bndbox']['ymax'])
				ymin = int(objs[i]['bndbox']['ymin'])
				
				# 중심값, normalized된 값
				center_x = (xmax + xmin) / (2 * width)
				center_y = (ymax + ymin) / (2 * height)
				bbox_width = (xmax - xmin) / width
				bbox_height = (ymax - ymin) / height

				# class 이름을 index로
				class_idx = class_dir[objs[i]['name']]
				results.append([class_idx, center_x, center_y, bbox_width, bbox_height])
			
		return results

In [ ]:
print(obj_base_path)

In [ ]:
# # test annotations
# for label_filename in test_label_list:
# 	with open(f"{test_label_base_path}/{label_filename}.yml") as f:
# 		# yaml 파일에서 annotation 정보 로드받기
# 		annotations = yaml.load(f, Loader=yaml.FullLoader)		

# 		# 새로운 annotation 값 계산
# 		new_annotations = transform_annotation(annotations)

# 	# txt 형태의 annotation 파일 만들기
# 	with open(f"{obj_base_path}/{label_filename}.txt", "w") as f:
# 		for annotation in new_annotations:
# 			line = " ".join(map(str, annotation)) + "\n"
# 			f.write(line)

In [ ]:
# # train annotations
# for label_filename in train_label_list:
# 	with open(f"{train_label_base_path}/{label_filename}.yml") as f:
# 		# yaml 파일에서 annotation 정보 로드받기
# 		annotations = yaml.load(f, Loader=yaml.FullLoader)		

# 		# 새로운 annotation 값 계산
# 		new_annotations = transform_annotation(annotations)

# 	# txt 형태의 annotation 파일 만들기
# 	with open(f"{obj_base_path}/{label_filename}.txt", "w") as f:
# 		for annotation in new_annotations:
# 			line = " ".join(map(str, annotation)) + "\n"
# 			f.write(line)

In [ ]:
def create_yaml_to_txt_annototations(base_path, filename_list):
    for filename in filename_list:
        with open(f"{base_path}/{filename}.yml") as f:
            # yaml 파일에서 annotation 정보 로드받기
            annotations = yaml.load(f, Loader=yaml.FullLoader)		

            # 새로운 annotation 값 계산
            new_annotations = transform_annotation(annotations)

        # txt 형태의 annotation 파일 만들기
        with open(f"{obj_base_path}/{filename}.txt", "w") as f:
            for annotation in new_annotations:
                line = " ".join(map(str, annotation)) + "\n"
                f.write(line)

In [ ]:
create_yaml_to_txt_annototations(test_label_base_path, test_label_list)
create_yaml_to_txt_annototations(train_label_base_path, train_label_list)


# Custom cfg file

# obj.data & obj.names file

## obj.data

In [ ]:
print(darknet_path)
print(yolov4_base_path)

In [ ]:
with open(darknet_path+"/data/obj.data", "w") as f:
    lines = f"classes = {len(class_dir)}\n"
    lines += f"train = {darknet_path}/data/train.txt\n"
    lines += f"valid = {darknet_path}/data/test.txt\n"
    lines += f"names = {darknet_path}/data/obj.names\n"
    lines += f"backup = {yolov4_base_path}/training\n"
    f.write(lines)

## obj.names

In [ ]:
with open(darknet_path+"data/obj.names", "w") as f:
    lines = ""
    for name in class_dir.keys():
        lines += name + '\n'
    f.write(lines)

# darknet

In [ ]:
# %cd /home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet
# !find ./data -mindepth 1 ! -regex '^./data/labels(/.*)?' -delete

In [ ]:
# subprocess(['rm', '-rf', f'{darknet_path}/cfg'])
# subprocess(["mkdir", f"{darknet_path}/cfg"])

Firstly, clean the cfg & data folders. Delete all files inside the data folder except the labels folder which is required for writing label names on the detection boxes and also delete all files inside the cfg folder as we already have our custom config file in the yolov4 main folder.

Next, copy all the files:

    - Copy yolov4-custom.cfg to the darknet/cfg directory.
    - Copy obj.names, obj.data and the obj folder to the darknet/data directory.
    - Copy process.py to the darknet directory.

5(a) Copy the obj dataset folder so that it is now in /darknet/data/ folder

5(b) Copy your yolov4-custom.cfg file so that it is now in /darknet/cfg/ folder

5(c) Copy the obj.names and obj.data files so that they are now in /darknet/data/ folder

## download weights

In [ ]:
subprocess.call(f"cd {darknet_path}", shell=True)
!pwd

In [ ]:
download_weight = "wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137"
subprocess.call(list(download_weight.split()))

# darknet 설치
https://webnautes.tistory.com/1482

In [ ]:
%cd /home/erbl/Git/RealTime-Object-Detection/yolov4-train

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Makefile
```
GPU=1 
CUDNN=1 
CUDNN_HALF=0 
OPENCV=1 
AVX=0 
OPENMP=0 
LIBSO=1
```

In [ ]:
%cd /home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet

In [ ]:
!make

# Training

In [ ]:
print(darknet_path)

In [ ]:
%cd /home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet

In [ ]:
!./darknet detector train "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet/data/obj.data" "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet/cfg/yolov4-custom.cfg" "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet/yolov4.conv.137" -map

# Check mAP

In [ ]:
!./darknet detector map "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet/data/obj.data" "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet/cfg/yolov4-custom.cfg" "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/training/yolov4-custom_best.weights" -points 0

# Run detector on a live webcam

In [ ]:
!./darknet detector demo "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet/data/obj.data" "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/darknet/cfg/yolov4-custom.cfg" "/home/erbl/Git/RealTime-Object-Detection/yolov4-train/training/yolov4-custom_best.weights" -thresh 0.5